# Game 3 - Hospital Resources

Skillfully allocating medical resources during a pandemic can save lives. Unexpected and extreme medical events not only disrupt hospitals' normal routines and protocol, but strain resources such as trained staff, space and pharmaceuticals, in addition to PPE and life-saving medical equipment. Determining how to most effectively allocate these resources is paramount to assuring the wellbeing of Sailors far from port.

In this scenario, you will oversee the medical bay of the USS GHOST where COVID-19 has taken hold of the crew. Your task is to create an algorithm to deploy the ship's available resources in a manner that will ensure as many Sailors as possible recover from the virus.

## brainstorming:
- reinforcement learning?

In [302]:
import pandas as pd
import numpy as np

* +30 10 ventilators (reusable)
* +20 10 oxygen masks (reusable)
    * cannot be combined with ventilator
* +15 10 plasma
* +30 7 remdesivir
* +25 20 Dexamethasone
* +15 10 casirivimab
* +10 17 chloroquine
* 30 beds
    * bed boosts health 5 points
    * bed allocation required for other treatments
* final health score = final health + bed bonus + rate of decline + treatment bonuses
* penalty of -100 if any sailor final health is 0


In [362]:
sailors_df = pd.read_csv("day1data.csv", header=1, index_col=0)
# sailors_df.drop("Unnamed: 3", axis=1, inplace=True)
resource_list = ["Bed", "Ventilator", "Remdesivir", "Dexamethasone", "Plasma", "Casirivimab", "Supplemental oxygen", "Chloroquine"]
for resource in resource_list:
    sailors_df[resource] = 0
columns = ["resource", "count", "amt_used", "reusable", "health_bonus"]
data = np.array([["Bed", 30, 0, True, 5],
          ["Ventilator", 10, 0, True, 30],
          ["Supplemental oxygen", 10, 0, True, 20],
          ["Plasma", 10, 0, False, 15],
          ["Remdesivir", 7, 0, False, 30],
          ["Dexamethasone", 20, 0, False, 25],
          ["Casirivimab", 10, 0, False, 15],
          ["Chloroquine", 17, 0, False, 10]])
resources_df = pd.DataFrame(data=data, columns=columns)
for col in ["count", "amt_used", "health_bonus"]:
    resources_df[col] = pd.to_numeric(resources_df[col])
print(resources_df,"\n")
sailors_df

              resource  count  amt_used reusable  health_bonus
0                  Bed     30         0     True             5
1           Ventilator     10         0     True            30
2  Supplemental oxygen     10         0     True            20
3               Plasma     10         0    False            15
4           Remdesivir      7         0    False            30
5        Dexamethasone     20         0    False            25
6          Casirivimab     10         0    False            15
7          Chloroquine     17         0    False            10 



,Health,Rate of health decline,Bed,Ventilator,Remdesivir,Dexamethasone,Plasma,Casirivimab,Supplemental oxygen,Chloroquine
ID,,,,,,,,,,
1,100,-1,0,0,0,0,0,0,0,0
2,75,-1,0,0,0,0,0,0,0,0
3,65,-7,0,0,0,0,0,0,0,0
4,55,-25,0,0,0,0,0,0,0,0
5,33,-30,0,0,0,0,0,0,0,0
6,89,-50,0,0,0,0,0,0,0,0
7,90,-60,0,0,0,0,0,0,0,0
8,45,-65,0,0,0,0,0,0,0,0
9,100,-79,0,0,0,0,0,0,0,0


In [379]:
def use_resource(sailors_df, resources_df, resource, indices):
    """
    resource = resource name as string
    indices = Pandas index list
    Ex. for single indices do [df.index[position]]
    Ex. for range of values do df.index[start:end]
    """
    bonus_dict = {}
    for r, bonus in zip(resources_df["resource"], resources_df["health_bonus"]):
        bonus_dict[r] = int(bonus)

    sailors_df.loc[indices, resource] = bonus_dict[resource]
    sailors_df.loc[indices, "Total"] += bonus_dict[resource]
    resources_df.loc[resources_df["resource"] == resource, "amt_used"] = len(indices)
    resources_df.loc[resources_df["resource"] == resource, "count"] -= len(indices)

def allocate_reusable(sailors_df, resources_df):    
    add_beds = sailors_df.index[:30]
    use_resource(sailors_df, resources_df, "Bed", add_beds)
    
    add_ventilators = sailors_df.index[:10]
    use_resource(sailors_df, resources_df, "Ventilator", add_ventilators)
    
    oxymask_candidates = sailors_df[(sailors_df["Ventilator"]==0) & (sailors_df["Bed"]>0)]
    add_oxymask = oxymask_candidates.index[:10]
    use_resource(sailors_df, resources_df, "Supplemental oxygen", add_oxymask)

def ded(sailors_df, resources_df):
    minarray = {}
    dying = sailors_df.loc[sailors_df["Total"] < 0]
    nonreusable = resources_df[(resources_df["reusable"]=="False") & resources_df["count"] > 0]
    for i in range(len(dying)):
        y = dying.iloc[i]["Total"]
        for j in range(len(nonreusable)):
            x = y + nonreusable.iloc[j]["health_bonus"]
            minarray[x] = nonreusable.iloc[j]["resource"]
        m = min(k for k in minarray.keys() if k > 0)
        resource = minarray[m]
        use_resource(sailors_df, resources_df, resource, [sailors_df.index[i]])
        minarray = {}

def allocation(sailors_df, resources_df):
    resources_df = resources_df.copy()
    sailors_df = sailors_df.copy()
    sailors_df["Total"] = sailors_df["Health"]+sailors_df["Rate of health decline"]
    sailors_df.sort_values(by=["Total"], inplace=True)
    
    allocate_reusable(sailors_df, resources_df)
    ded(sailors_df, resources_df)
    
    return sailors_df, resources_df

def calc_final_score(allocation_df):
    final_health = allocation_df["Total"].apply(lambda x: 100 if x > 100 else x)
    final_health = final_health.apply(lambda x: -100 if x < 1 else x)
    return final_health.sum()


In [380]:
a_df, r_df = allocation(sailors_df, resources_df)
a_df

,Health,Rate of health decline,Bed,Ventilator,Remdesivir,Dexamethasone,Plasma,Casirivimab,Supplemental oxygen,Chloroquine,Total
ID,,,,,,,,,,,
20,20,-80,5,30,30,0,0,0,0,0,5
15,39,-89,5,30,30,0,0,0,0,0,15
14,28,-67,5,30,0,0,0,0,0,10,6
8,45,-65,5,30,0,0,0,0,0,0,15
13,55,-68,5,30,0,0,0,0,0,0,22
11,60,-62,5,30,0,0,0,0,0,0,33
5,33,-30,5,30,0,0,0,0,0,0,38
10,95,-88,5,30,0,0,0,0,0,0,42
9,100,-79,5,30,0,0,0,0,0,0,56
